In [ ]:
##GET PREDICTIONS OF ALL MODELS IN A PORTFOLIO



In [ ]:
for ticker in tickersTraded:
    netPosition = 0.0
    for pred in portfolio.getModelPrediction(ticker):
        ##CHECK IF PREDICTION STILL VALID
        if len(joinedData[str(pred["lastDataDayUsed"]):]) - 1 < pred["predictionLength"]:##GETS TRADING DAYS SINCE LAST DATA DAY
            ##MUST DO -1 BECAUSE INCLUDES DAY LAST USED IN LEN
#             print(pred["ticker"], pred["prediction"])
            netPosition += pred["prediction"]
    print(ticker, netPosition)